In [219]:
import pandas as pd
import numpy as np
from datetime import datetime
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource
import bokeh
output_notebook()

Loading BokehJS ...

In [4]:
#read file
top200 = pd.read_csv('BB200.csv')

In [201]:
catalog = pd.read_csv('BBCatalog.csv')

In [5]:
#drop duplicates
print(len(top200))
top200 = top200.drop_duplicates()
print(len(top200))

524287
524131


In [204]:
print(len(catalog))
catalog = catalog.drop_duplicates()
print(len(catalog))

70750
70748


In [6]:
#convert date column from string to date
#top200.date=pd.to_datetime(top200.date)
d = np.asarray([datetime.strptime(x,'%m/%d/%Y').date() for x in top200.date])
top200.date = d

In [205]:
d = np.asarray([datetime.strptime(x,'%m/%d/%Y').date() for x in catalog.date])
catalog.date = d

In [208]:
catalog.head()

,album,artist,date,rank,Artist_Album
0,Mamma Mia!,Soundtrack,2018-08-04,1,Soundtrack-Mamma Mia!
1,Gold -- Greatest Hits,ABBA,2018-08-04,2,ABBA-Gold -- Greatest Hits
2,Greatest Hits,Queen,2018-08-04,3,Queen-Greatest Hits
3,The Essential Michael Jackson,Michael Jackson,2018-08-04,4,Michael Jackson-The Essential Michael Jackson
4,Legend: The Best Of...,Bob Marley And The Wailers,2018-08-04,5,Bob Marley And The Wailers-Legend: The Best Of...


In [8]:
#create new artist_album ID column
top200['Artist_Album'] = top200.artist + '-' + top200.album

In [207]:
catalog['Artist_Album']=catalog.artist+'-'+catalog.album

#####################
#  TOP 200 ANALYSIS #
#####################

In [158]:
#albums by weeks at 1
topalbums = top200[top200['rank']==1].groupby('Artist_Album').agg({'date':'count'})\
                          .sort_values(by='date',ascending = False)[:10].reset_index()

In [185]:
#chart for above
source = ColumnDataSource(topalbums)
p = figure(x_range=list(topalbums['Artist_Album']), plot_height=500, title='Weeks at Number 1')
p.vbar(x='Artist_Album',top='date', source=source, width=0.9,color='darkred')
p.xaxis.major_label_orientation = -1/3
p.yaxis.axis_label = "Weeks"
show(p)

In [187]:
#albums that have have spent the most weeks on the chart
topdwellers = top200.groupby(['Artist_Album']).agg({'date':'count'})\
.sort_values(by='date',ascending = False)[:10].reset_index()

In [190]:
#chart for above
source = ColumnDataSource(topdwellers)
p = figure(x_range=list(topdwellers['Artist_Album']), plot_height=500, title='Total Weeks on Chart')
p.vbar(x='Artist_Album',top='date', source=source, width=0.9,color='darkred')
p.xaxis.major_label_orientation = -1/3
p.yaxis.axis_label = "Weeks"
show(p)

In [198]:
#artists with most entries reaching number 1
plot0 = top200[top200['rank']==1].groupby('artist').agg({'album':'nunique'})\
.sort_values(by='album',ascending=False)[2:12].reset_index()

In [199]:
#plot for above
source = ColumnDataSource(plot0)
p = figure(x_range=list(plot0['artist']), title = "Most Albums to Reach #1",plot_height=500)
p.vbar(x='artist', top='album', source=source, width=0.9, color='darkred')
p.xaxis.major_label_orientation = -1/3
show(p)

In [54]:
#artists with most entries on top200
plot1 = top200.groupby('artist').agg({'album':'nunique','date':['min','max'],'rank':'mean'})\
.sort_values([('album','nunique')],ascending=False)[2:12]

In [192]:
#plot for above
p = figure(x_range=artists1, title = "Artists with the most entries on Top 200",plot_height=500)
p.vbar(x=artists1, top=list(plot1.album['nunique']), width=0.9,color='darkred')
p.xaxis.major_label_orientation = -1/3
show(p)

In [200]:
len(top200[top200['artist']=='The Beatles'].groupby('album').agg('count'))

48

In [237]:
top200[top200['album']=='Thriller'].sort_values(by='date')
source = ColumnDataSource(top200[top200['album']=='Thriller'].sort_values(by='date')[:104])
p = figure(y_range=(200,1), plot_width=500, plot_height=400,x_axis_type="datetime", title="Thriller: First 2 Years")
p.circle('date','rank',source=source,size=3,color='black')
p.line('date','rank',source=source,color='darkred')
p.yaxis.axis_label = "Chart Ranking"
show(p)

In [234]:
top200[top200['album']=='The Dark Side Of The Moon'].sort_values(by='date')
source = ColumnDataSource(top200[top200['album']=='The Dark Side Of The Moon'].sort_values(by='date')[:104])
p = figure(y_range=(200,1), plot_width=500, plot_height=400,x_axis_type="datetime", title="Dark Side Of The Moon: First 2 Years")
p.circle('date','rank',source=source,size=3,color='black')
p.line('date','rank',source=source,color='darkred')
p.yaxis.axis_label = "Chart Ranking"
show(p)

####################
# CATALOG ANALYSIS #
####################

In [242]:
#most appearances as 1 on catalog
plot2 = catalog[catalog['rank']==1].groupby('Artist_Album').agg({'date':'count'})\
.sort_values('date',ascending=False)[:10].reset_index()

In [252]:
source = ColumnDataSource(plot2)
p = figure(x_range=list(plot2['Artist_Album']), title = "Most Weeks at 1 on Catalog Chart",plot_height=500)
p.vbar(x='Artist_Album', top='date', source=source, width=0.9, color='darkred')
p.xaxis.major_label_orientation = -1/3
p.yaxis.axis_label="Weeks"
show(p)

In [250]:
#most appearances on catalog chart
plot3 = catalog.groupby('Artist_Album').agg({'date':'count'}).sort_values('date',ascending=False)[:10].reset_index()

In [261]:
source = ColumnDataSource(plot3)
p = figure(x_range=list(plot3['Artist_Album']), title = "Most Weeks on Catalog Chart",plot_height=500)
p.vbar(x='Artist_Album', top='date', source=source, width=0.9, color='darkred')
p.xaxis.major_label_orientation = -1/3
p.yaxis.axis_label="Weeks"
show(p)

##################
# CROSS ANALYSIS #
##################

In [266]:

#t200 = top200.groupby('Artist_Album').agg({'album':'count','date':['min','max'],'rank':['min','max']})
joined = top200.merge(catalog, on='Artist_Album', how='inner')
len(joined.groupby('Artist_Album')\
.agg({'album_x':'count','rank_x':['min','max'],'date_x':['min','max'],'rank_y':['min','max'],'date_y':['min','max']})\
.sort_values([('album_x','count')],ascending=True))

2020

In [281]:
antij = pd.merge(top200, catalog, on='Artist_Album', how='outer', indicator=True).query('_merge == "right_only"')

4077

In [ ]:
antij = antij.groupby('Artist_Album').agg({'date_y':['count','min','max'],'rank_y':['min','max','mean']})\
.sort_values([('date_y','count')],ascending=False)

In [310]:
len(antij[antij['date_y']['max']>datetime(2010,1,1).date()])

67

In [296]:
antij.head()

date_y              \
                                                    count         min   
Artist_Album                                                            
Patsy Cline-12 Greatest Hits                          290  1991-05-25   
Lynyrd Skynyrd-Skynyrd's Innyrds/Their Greatest...    198  1991-07-27   
Miles Davis-Kind Of Blue                              173  1997-06-14   
Poison-Greatest Hits 1986--1996                       124  1999-02-06   
The Police-Every Breath You Take - The Singles        110  1991-06-08   

                                                               rank_y        \
                                                           max    min   max   
Artist_Album                                                                  
Patsy Cline-12 Greatest Hits                        2003-11-15    6.0  50.0   
Lynyrd Skynyrd-Skynyrd's Innyrds/Their Greatest...  1999-02-27   11.0  50.0   
Miles Davis-Kind Of Blue                            2015-10-24    2.0  50.0   
Poison-Greatest Hits 1986--1996                     2005-05-07    2.0  50.0   
The Police-Every Breath You Take - The Singles      1994-02-12   23.0  50.0   

                                                               
                                                         mean  
Artist_Album                                                   
Patsy Cline-12 Greatest Hits                        26.868966  
Lynyrd Skynyrd-Skynyrd's Innyrds/Their Greatest...  35.323232  
Miles Davis-Kind Of Blue                            32.069364  
Poison-Greatest Hits 1986--1996                     36.250000  
The Police-Every Breath You Take - The Singles      35.536364

In [275]:
top200.Artist_Album.nunique()

34486

In [276]:
catalog.Artist_Album.nunique()

2368

In [ ]:
# there should be 2368-2020 = 348 unique albums on the anti-join

In [279]:
2020/2368*100

85.30405405405406

In [ ]:
# IDEA: create a table 

In [68]:
#albums by number of weeks within top 10
top200[top200['rank']<=10].groupby("Artist_Album").agg({'date':'count'}).sort_values(by='date',ascending = False)[:10]

,date
Artist_Album,
Bruce Springsteen-Born In The U.S.A.,84
Adele-21,82
Michael Jackson-Thriller,78
Def Leppard-Hysteria,78
Alanis Morissette-Jagged Little Pill,72
Paula Abdul-Forever Your Girl,64
Celine Dion-Falling Into You,61
Taylor Swift-1989,59
Lionel Richie-Can't Slow Down,59


In [85]:
#artists and their peak rank
chartboys=top200.groupby("artist").agg({'album':'nunique','rank':'min'}).sort_values('album',ascending=False)
#proportion of artists who have at least 1 album peak: 4.9%
print(len(chartboys[chartboys['rank']==1])/len(chartboys)*100)
#proportion of artists with at least 5 entries and did rank 1: ~19%
chartboys[(chartboys['album']>=5) & (chartboys['rank']==1)]
len(chartboys[(chartboys['album']>=5) & (chartboys['rank']==1)]) / len(chartboys[chartboys['album']>=5])*100
# IDEA: create a table with 3 columns: min no. of entries on poll, avg of artists that peaked at 1,avg to make top 10
# IDEA: create a table with 3 columns: exact no. of entries on poll, avg of artists that peaked at 1, avg to make top 10

4.8915945743885505


0.18970047293746717

In [100]:
#albums and their peak rank
chartgirls=top200.groupby("Artist_Album").agg({'rank':['min','count']}).sort_values([('rank','min')],ascending=True)
# avg number of weeks on the charts for an album that peaks at 1: 64
chartgirls[chartgirls[('rank','min')]==1][('rank','count')].mean()
# avg number of weeks at number 1: 2.623
top200[top200['rank']==1].groupby('Artist_Album').agg({'date':'count'}).sort_values('date',ascending=False).mean()

date    2.623377
dtype: float64

In [123]:
a=list(chartgirls[chartgirls[('rank','min')]==1].index)
#only the albums that have reached number 1
top200[top200['Artist_Album'].isin(a)]
#weeks on chart for albums that have reached 1
top200[top200['Artist_Album'].isin(a)].groupby('Artist_Album').agg({'artist':'min','date':'count'}).sort_values(by='date',ascending=False)

,artist,date
Artist_Album,,
Pink Floyd-The Dark Side Of The Moon,Pink Floyd,927
Metallica-Metallica,Metallica,491
Eminem-Curtain Call: The Hits,Eminem,399
Nirvana-Nevermind,Nirvana,397
Adele-21,Adele,370
Eminem-The Eminem Show,Eminem,352
Michael Jackson-Thriller,Michael Jackson,331
The Beatles-1,The Beatles,326
Carole King-Tapestry,Carole King,318


In [128]:
len(top200['date'].unique())

2626